In [2]:
import sys
import os

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

from run import main

# Finds thresholds for IH-confidence and IH-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/heart",
    train_name="heart_train",
    measure="ih",  # or "if"
    metric="macro-f1",
    steps=[#"build", "results", "reports", "plot", 
        "grid", "heuristic", #"brute", "annealing" 
          ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.99375
Rejecting threshold(s): 0.74
Associated minimum cost: 0.602778

Uncertainty-based rejection
Filtering threshold(s): 0.99375
Rejecting threshold(s): 0.58
Associated minimum cost: 0.713054


In [3]:
# Finds thresholds for IF-confidence and IF-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/heart",
    train_name="heart_train",
    measure="if",  # or "if"
    metric="macro-f1",
    steps=[#"build",
           #"results", "reports", 
           #"plot", 
           "grid", "heuristic", #"brute", "annealing"
    ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 1.0
Rejecting threshold(s): 0.74
Associated minimum cost: 0.714755

Uncertainty-based rejection
Filtering threshold(s): 1.0
Rejecting threshold(s): 0.56
Associated minimum cost: 0.748481


In [4]:
from thresholds_tester import ThresholdsTester

import sys
import os

# Test the thresholds found in the test set

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

tester = ThresholdsTester(
    data_path="(...)/RefineAI/experiments/heart",
    train_name="heart_train",
    test_name="heart_test",
    metric="macro-f1",
    ih_uncertainty=[0.99375, 0.58], #filtering, rejecting
    ih_confidence=[0.99375, 0.74],
    if_uncertainty=[1, 0.56],
    if_confidence=[1, 0.74])

tester.final_measures()
final, macro = tester.final_results()

In [5]:
# Reorganizes the results 

final = final.reindex([2, 1, 0, 5, 4, 3, 8, 7, 6, 11, 10, 9, 12, 13]) 
final

,measure,metric,T_value,thresholds,precision_class_0,precision_class_1,recall_class_0,recall_class_1,rate_accep_0,rate_accep_1,mean_conf_uncert,f1_0,f1_1,f1-macro,acp_a
2,ih,C,0.000,0.74,0.50,0.986,0.75,0.958,0.8,0.845,0.910,0.600000,0.971798,0.785899,0.8225
1,ih,C,0.006,0.74,0.60,0.975,0.60,0.975,1.0,0.952,0.940,0.600000,0.975000,0.787500,0.9760
0,ih,C,0.006,0.00,0.60,0.976,0.60,0.976,1.0,1.000,0.926,0.600000,0.976000,0.788000,1.0000
5,ih,U,0.000,0.58,1.00,0.984,0.75,1.000,0.8,0.750,0.712,0.857143,0.991935,0.924539,0.7750
4,ih,U,0.006,0.58,1.00,0.986,0.75,1.000,0.8,0.833,0.793,0.857143,0.992951,0.925047,0.8165
3,ih,U,0.006,0.00,0.60,0.976,0.60,0.976,1.0,1.000,0.747,0.600000,0.976000,0.788000,1.0000
8,if,C,0.000,0.74,0.50,0.986,0.75,0.958,0.8,0.845,0.910,0.600000,0.971798,0.785899,0.8225
7,if,C,0.000,0.74,0.50,0.986,0.75,0.958,0.8,0.845,0.910,0.600000,0.971798,0.785899,0.8225
6,if,C,0.000,0.00,0.25,0.974,0.60,0.893,1.0,1.000,0.864,0.352941,0.931743,0.642342,1.0000
11,if,U,0.000,0.56,1.00,0.985,0.75,1.000,0.8,0.786,0.706,0.857143,0.992443,0.924793,0.7930


In [6]:
# Calculates the cost for each solution, used to statistical comparison

macro = macro[['measure', 'metric', 'f1-macro','acp_a','mean_conf_uncert']]
macro = round(macro,3)

macro['cost'] = 4*(1-macro['f1-macro']) + (1-(macro['acp_a'])) + (1-macro['mean_conf_uncert'])

macro_confidence = macro.reindex([1, 7, 12])
macro_uncertainty = macro.reindex([4, 10, 13])

print(','.join([f"'{x:.3}'" for x in macro_uncertainty.iloc[:,-1]]))

'0.691','0.801','1.76'


In [7]:
print(','.join([f"'{x:.3}'" for x in macro_confidence.iloc[:,-1]]))

'0.932','1.12','1.57'
